# Functions to Update Alert Database

# Prep

## Import Packages

In [1]:
# File Manipulation

import os # For working with Operating System
import sys # System arguments
from dotenv import load_dotenv # Loading .env info

# Web

import requests # Accessing the Web

# Time

import datetime as dt # Working with dates/times
import pytz # Timezones

# Database 

import psycopg2
from psycopg2 import sql

# Data Manipulation

import numpy as np
import geopandas as gpd
import pandas as pd

### Load Functions

In [4]:
script_path = os.path.join('..', '..', 'Scripts', 'python')

# Function definition - Please see Scripts/python/*
exec(open(os.path.join(script_path, 'Get_spikes_df.py')).read())
exec(open(os.path.join(script_path, 'Send_Alerts.py')).read())

### Global Variables

In [3]:
load_dotenv() # Load .env file

## API Keys

purpleAir_api = os.getenv('PURPLEAIR_API_TOKEN') # PurpleAir API Read Key

## Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))  

# Other Constants from System Arguments

spike_threshold = 35 # Value which defines an AQ_Spike (Micgrograms per meter cubed)

timestep = 10 # Sleep time in between updates (in Minutes)

# When to stop the program? (datetime)
days_to_run = 7 # How many days will we run this?
stoptime = dt.datetime.now() + dt.timedelta(days=days_to_run)

## Compute/Define other necessary variables 

In [6]:
#  Get the sensor_ids from sensors in our database

sensor_ids = get_sensor_ids(pg_connection_dict) # In Get_Spikes_df.py

In [7]:
# Get Spikes Dataframe and runtime

spikes_df, runtime, flagged_sensor_ids = Get_spikes_df(purpleAir_api, sensor_ids, spike_threshold) # In Get_Spikes_df.py

# Definitions

## Initial Functions

### Get Active Alerts

In [8]:
# Get active alerts from database

conn = psycopg2.connect(**pg_connection_dict)

# Create json cursor
cur = conn.cursor()

cmd = sql.SQL('''SELECT * 
FROM "Active Alerts Acute PurpleAir"
''')

cur.execute(cmd) # Execute

conn.commit() # Committ command

# Convert response into dataframe

cols_for_active_alerts = ['alert_index', 'sensor_indices', 'start_time', 'max_reading']
active_alerts = pd.DataFrame(cur.fetchall(), columns = cols_for_active_alerts)

# Close cursor
cur.close()
# Close connection
conn.close()

In [9]:
active_alerts

,alert_index,sensor_indices,start_time,max_reading
0,64,[145614],2023-11-03 11:59:01,92.8


### Sort the Sensor IDs

In [10]:
# Check for 4 types of Sensor ID
# Using set operations between:

# Currently active
current_active_spike_sensors = set(spikes_df.sensor_index) # From most recent api call

# Previously active
if len(active_alerts) > 0:
    previous_active_spike_sensors = set(active_alerts.sensor_indices.sum()) # From our database
    # The sensor_indices are given as lists of indices because we may cluster alerts eventually
else:
    previous_active_spike_sensors = set()

# The sets:

# 1) new
new_spike_sensors = current_active_spike_sensors - previous_active_spike_sensors

# 2) ongoing, 
ongoing_spike_sensors = current_active_spike_sensors.intersection(previous_active_spike_sensors)

# 3) ended alerts
ended_spike_sensors = previous_active_spike_sensors - current_active_spike_sensors

# 4) Not Spiked
not_spiked_sensors = set(sensor_ids.astype(int)) - current_active_spike_sensors - set(flagged_sensor_ids.astype(int))

In [11]:
new_spike_sensors

set()

In [12]:
ongoing_spike_sensors

set()

In [13]:
ended_spike_sensors

{145614}

## New Alerts

For each new alert, we should:

In [36]:
# Initialize for For Loop

# Iterable (Find rows in spikes_df that are new spikes)

new_spikes = spikes_df[spikes_df.sensor_index.isin(new_spike_sensors)]

### Add to Active Alerts

In [28]:
# 1) Add to active alerts

def add_to_active_alerts(row, pg_connection_dict, runtime_for_db):
    '''
    This takes a row from spikes_df[spikes_df.sensor_index.isin(new_spike_sensors)],
    the connection dictionary,
    runtime_for_db = when purpleair was queried as a string (runtime.strftime('%Y-%m-%d %H:%M:%S'))
    
    it returns the alert_index that it created
    
    '''

    cols_for_db = ['sensor_indices', 'start_time', 'max_reading']
    sensor_index = row.sensor_index
    reading = row.pm25

    # 1) Add to active alerts

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    # This is really a great way to insert a lot of data

    vals = [[sensor_index], runtime_for_db, reading]
    
    q1 = sql.SQL('INSERT INTO "Active Alerts Acute PurpleAir" ({}) VALUES ({});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db))))

    cur.execute(q1.as_string(conn),
        (vals)
        )
    # Commit command
    conn.commit()

    # Get the alert_index that was just created
    
    cmd = sql.SQL('''SELECT alert_index
FROM "Active Alerts Acute PurpleAir"
WHERE sensor_indices = {}::int[];'''
             ).format(sql.Literal([sensor_index]))
    
    cur.execute(cmd)     
    
    conn.commit() # Committ command
    
    newest_alert_index = cur.fetchall()[0][0]

    # Close cursor
    cur.close()
    # Close connection
    conn.close()

    return newest_alert_index

### Update User's Active Alerts

Add these alerts to user's active_alerts in database

In [26]:
def update_users_active_alerts(record_ids, alert_index, pg_connection_dict):
    '''
    This function takes a list of record_ids (users), an alert index (integer), and pg_connection_dict

    It will add this alert index to all the record_ids' active_alerts
    '''

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
UPDATE "Sign Up Information"
SET active_alerts = ARRAY_APPEND(active_alerts, {}) -- inserted alert_index
WHERE record_id = ANY ( {} ); -- inserted record_ids 
    ''').format(sql.Literal(alert_index),
                sql.Literal(record_ids)
               )

    cur.execute(cmd
        )
    # Commit command
    conn.commit()

    # Close cursor
    cur.close()
    # Close connection
    conn.close()

### For loop for Experimenting

In [34]:
# The For Loop

# Fake values
new_spike_sensors = [142720]
new_spikes_df = pd.DataFrame([[142720,35.]], columns = ['sensor_index', 
                               'pm25'])


if len(new_spike_sensors) > 0:

    # new_spikes_df = spikes_df[spikes_df.sensor_index.isin(new_spike_sensors)] 

    for index, row in new_spikes_df.iterrows():

        # 1) Add to active alerts
    
        newest_alert_index = add_to_active_alerts(row, pg_connection_dict,
                             runtime.strftime('%Y-%m-%d %H:%M:%S') # When we ran the PurpleAir Query
                            ) # In Update_Alerts.py
        
        # 2) Query users ST_Dwithin 1000 meters & subscribed = TRUE
        
        record_ids_nearby = Users_nearby_sensor(pg_connection_dict, row.sensor_index, 1000) # in Send_Alerts.py
        
        if len(record_ids_nearby) > 0:

            # if (now.hour < too_late_hr) & (now.hour > too_early_hr): # Waking Hours
        
            #     # a) Query users from record_ids_nearby if both active_alerts and cached_alerts are empty
            #     record_ids_new_alerts = Users_to_message_new_alert(pg_connection_dict, record_ids_nearby) # in Send_Alerts.py & .ipynb 
                
            #     # Compose Messages & concat to messages/record_id_to_text   
                
            #     # # Add to message/record_id storage for future messaging
            #     # record_ids_to_text += record_ids_new_alerts
            #     # messages += [new_alert_message(sensor_id)]*len(record_ids_new_alerts) # in Compose_Messages.py
                
            # b) Add newest_alert_index to record_ids_nearby's Active Alerts
            update_users_active_alerts(record_ids_nearby, newest_alert_index, pg_connection_dict) # in Update_Alerts.py & .ipynb

## Ongoing Alerts

In [41]:
ongoing_spike_sensors

{142718,
 142720,
 142724,
 142726,
 142738,
 142750,
 143634,
 143942,
 145454,
 145470,
 145506,
 157861,
 157935}

In [42]:
# Initialize for For Loop

# Iterable (Find rows in spikes_df that are new spikes)

ongoing_spikes = spikes_df[spikes_df.sensor_index.isin(ongoing_spike_sensors)]

ongoing_spikes

,sensor_index,pm25
0,142718,43.8
1,142720,42.8
2,142726,37.6
3,142724,41.5
4,142738,45.0
5,142750,36.0
7,143634,35.8
9,143942,35.8
10,145454,42.2
11,145470,52.1


In [43]:
# Function Definitions

# For each ongoing alert, we should

# 1) Update max_reading if it's higher

def update_max_reading(row, pg_connection_dict):
    '''
    Row should be a row from the ongoing_spikes dataFrame
    '''

    sensor_index = row.sensor_index
    reading = row.pm25

    # 1) Add to active alerts

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
UPDATE "Active Alerts Acute PurpleAir"
SET max_reading = GREATEST({}, max_reading)
WHERE {} = ANY (sensor_indices);

''').format(sql.Literal(reading), sql.Literal(sensor_index))

    cur.execute(cmd)
    # Commit command
    conn.commit()

    # Close cursor
    cur.close()
    # Close connection
    conn.close()

In [44]:
# The For Loop

if len(ongoing_spikes) > 0:

    for _, spike in ongoing_spikes.iterrows():
    
        update_max_reading(spike, pg_connection_dict)

## Ended Alerts

In [45]:
# # # To manufacture an ended alert

# new_spike_sensors = new_spike_sensors - {143944}
# ended_spike_sensors = {143944}

In [46]:
# print(not_spiked_sensors)

In [47]:
# Function Definitions

# For each ended alert, we should

# 1) Add to archived alerts

def add_to_archived_alerts(not_spiked_sensors, pg_connection_dict):
    '''
    '''

    # Get relevant sensor indices as list
    sensor_indices = list(not_spiked_sensors)
                           
    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()

    # This command selects the ended alerts from active alerts
    # Then it gets the difference from the current time and when it started
    # Lastly, it inserts this selection while converting that time difference into minutes for duration_minutes column
    cmd = sql.SQL('''
    WITH ended_alerts as
    (
    	SELECT alert_index, sensor_indices, start_time, CURRENT_TIMESTAMP AT TIME ZONE 'America/Chicago' - start_time as time_diff, max_reading 
     	FROM "Active Alerts Acute PurpleAir"
     	WHERE sensor_indices <@ {}::int[] -- contained
    )
    INSERT INTO "Archived Alerts Acute PurpleAir" 
    SELECT alert_index, sensor_indices, start_time, (((DATE_PART('day', time_diff) * 24) + 
    	DATE_PART('hour', time_diff)) * 60 + DATE_PART('minute', time_diff)) as duration_minutes, max_reading
    FROM ended_alerts;
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    # Close cursor
    cur.close()
    # Close connection
    conn.close()
    

#~~~~~~~~~~~~~~~~

In [48]:
# 2) Remove from active alerts

def remove_active_alerts(not_spiked_sensors, pg_connection_dict):
    '''
    This function removes the ended_spikes from the Active Alerts Table
    It also retrieves their alert_index
    
    ended_spike_sensors is a set of sensor indices that have ended spikes Alerts
    
    ended_alert_indices is returned alert_index of the removed alerts for accessing Archive for end message 
    
    '''

    # Get relevant sensor indices as list
    sensor_indices = list(not_spiked_sensors)
                           
    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
    SELECT alert_index
    FROM "Active Alerts Acute PurpleAir"
    WHERE sensor_indices <@ {}::int[]; -- contained
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    ended_alert_indices = [i[0] for i in cur.fetchall()]
    
    cmd = sql.SQL('''
    DELETE FROM "Active Alerts Acute PurpleAir"
    WHERE sensor_indices <@ {}::int[]; -- contained
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    # Close cursor
    cur.close()
    # Close connection
    conn.close()

    return ended_alert_indices

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
  

In [49]:
# To test

if len(ended_spike_sensors) > 0:

    add_to_archived_alerts(not_spiked_sensors.union(ended_spike_sensors), pg_connection_dict) # Add the ended SpikeAlerts to archive
    
    ended_alert_indices = remove_active_alerts(not_spiked_sensors.union(ended_spike_sensors), pg_connection_dict) # Remove them from Active Alerts

    print(ended_alert_indices)

[]


In [41]:
  
# 3) Transfer these alerts from "Sign Up Information" active_alerts to "Sign Up Information" cached_alerts

def cache_alerts(ended_alert_indices, pg_connection_dict):
    '''
    This function transfers a list of ended_alert_indices from "Sign Up Information" active_alerts to "Sign Up Information" cached_alerts
    '''
    
    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    for alert_index in ended_alert_indices:
    
        cmd = sql.SQL('''
        UPDATE "Sign Up Information"
        SET active_alerts = ARRAY_REMOVE(active_alerts, {}), -- Inserted alert_index
            cached_alerts = ARRAY_APPEND(cached_alerts, {}) -- Inserted alert_index
        WHERE {} = ANY (active_alerts);
        ''').format(sql.Literal(alert_index),
                    sql.Literal(alert_index),
                    sql.Literal(alert_index)
                   )
        cur.execute(cmd)
    # Commit command
    conn.commit()

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
   

In [ ]:
 
# 4-5b) Query for people to text, initialize reports, text people - see Send_Alerts.py

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    

In [ ]:
# 5c) Clear a users' cache

def clear_cached_alerts(record_ids, pg_connection_dict):
    '''
    This function clears the cached_alerts for all users with the given record_ids (a list of integers)
    '''

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
    UPDATE "Sign Up Information"
    SET cached_alerts = {} 
    WHERE record_id = ANY ( {} );
    ''').format(sql.Literal('{}'),
                sql.Literal(record_ids)
               )
    
    cur.execute(cmd)
    # Commit command
    conn.commit()